In [2]:
from datasets import load_dataset

In [36]:
from datasets import Split, Dataset, DatasetDict

raw_dataset: Dataset = load_dataset('text', data_files='legal_mlm_bert/mini_bert_corpus_path.txt', split=Split.TRAIN)
split_raw_dataset: DatasetDict = raw_dataset.train_test_split(test_size=0.1)

Using custom data configuration default-29c505684d507cec


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /Users/LongNH/.cache/huggingface/datasets/text/default-29c505684d507cec/0.0.0/d86c40dad297bdddf277b406c6a59f0250b5318c400bf23d420a31aff88c84c4. Subsequent calls will reuse this data.


In [85]:
%%capture
from transformers import AutoTokenizer, RobertaTokenizer, PhobertTokenizer

tokenizer = PhobertTokenizer.from_pretrained("vinai/phobert-base")


def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["text"]])

loading file https://huggingface.co/vinai/phobert-base/resolve/main/vocab.txt from cache at /Users/LongNH/.cache/huggingface/transformers/970c6224b2713c8b52a7bcfc4d5a951c9bb88302e4523388b50f28284e87ac44.26ba0c8945e559c68d0bc35d24fea16f5463a49fe8f134e0c32261d590b577fa
loading file https://huggingface.co/vinai/phobert-base/resolve/main/bpe.codes from cache at /Users/LongNH/.cache/huggingface/transformers/f3a66ae0a78d1a53b3eb99e31837d0d8e2f684a2dcc1f52f75fd36873e3d79de.301ac8958de708ddcea8500d9acbe6261dba391d249c98dcda1e49dbbff870dd
loading file https://huggingface.co/vinai/phobert-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/vinai/phobert-base/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/vinai/phobert-base/resolve/main/tokenizer_config.json from cache at None
loading file https://huggingface.co/vinai/phobert-base/resolve/main/tokenizer.json from cache at None
loading configuration file https://hu

In [86]:
type(tokenizer)

transformers.models.phobert.tokenization_phobert.PhobertTokenizer

In [87]:
tokenized_dataset = split_raw_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=split_raw_dataset["train"].column_names,
)

Token indices sequence length is longer than the specified maximum sequence length for this model (774 > 256). Running this sequence through the model will result in indexing errors
num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


  0%|          | 0/1 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (713 > 256). Running this sequence through the model will result in indexing errors


In [88]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'token_type_ids'],
        num_rows: 4
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'token_type_ids'],
        num_rows: 1
    })
})

In [89]:
block_size = 128

def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    result = {
        k: [t[i: i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result


lm_dataset = tokenized_dataset.map(group_texts, batched=True, num_proc=4)

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


  0%|          | 0/1 [00:00<?, ?ba/s]

In [90]:
lm_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
        num_rows: 13
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
        num_rows: 6
    })
})

In [68]:
from transformers import DataCollatorForLanguageModeling

# tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [93]:
len(tokenizer.pad(lm_dataset['train'][0], padding='max_length')['input_ids'])

256

In [69]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, IntervalStrategy

model = AutoModelForCausalLM.from_pretrained("vinai/phobert-base")

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy=IntervalStrategy.EPOCH,
    learning_rate=2e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

loading configuration file https://huggingface.co/vinai/phobert-base/resolve/main/config.json from cache at /Users/LongNH/.cache/huggingface/transformers/a596f267f08b7158c7ab6300b1bf98eb6e1b05e6bcb0d7c18a8070364ee3011b.bbe27b2cac909b2279c83792c2d2b6f159f0a95f5d1c1eb66451da1c89a53609
Model config RobertaConfig {
  "_name_or_path": "vinai/phobert-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 258,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "PhobertTokenizer",
  "transformers_version": "4.15.0",
  "type_vocab_size": 1,
  "use_c

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.